In [1]:
# count the number of csv files in the financial data folder
import os
import csv
import pandas as pd


In [2]:
# Corrected file names list without PG 
file_names = ['AAPL', 'AMGN', 'AMZN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'TSLA', 'UNH', 'Visa', 'VZ', 'WBA', 'WMT']

# Corrected folder path without the extra space
folder_path = '/Users/senamiruetschlin/Documents/Studies /2. Semester/Guided Studies in Financial Management/Group Project/Investor_Attention/financial_data/stocks'
def process_stock_data(stock_symbol, folder_path):
    file_path = os.path.join(folder_path, f"{stock_symbol}_financial.csv")
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        return None
    try:
        df_temp = pd.read_csv(file_path)
        df_temp['stock'] = stock_symbol
        cols = list(df_temp.columns)
        cols = [cols[-1]] + cols[:-1]
        df_temp = df_temp[cols]
        return df_temp
    except pd.errors.ParserError as e:
        print(f"ParserError: {e}")
        return None

# List to store dataframes for each stock
stock_dfs = []

# Process each stock's data
for name in file_names:
    stock_df = process_stock_data(name, folder_path)
    if stock_df is not None:
        stock_dfs.append(stock_df)

# Concatenate dataframes for all stocks into a single dataframe
combined_df = pd.concat(stock_dfs, ignore_index=True)

# Access the combined dataframe
combined_df


ParserError: Error tokenizing data. C error: Expected 13 fields in line 2793, saw 14



,stock,Date,PX_LAST,Change,% Change,PX_OPEN,Change.1,% Change.1,PX_HIGH,Change.2,% Change.2,PX_LOW,Change.3,% Change.3,PX_VOLUME
0,AAPL,2023-12-29,192.2850,-1.048,-0.542070,193.6530,0.320,0.165518,194.1520,0.819,0.423621,191.4810,-1.852,-0.957933,NaN
1,AAPL,2023-12-28,193.3330,0.429,0.222390,193.8930,0.989,0.512690,194.4120,1.508,0.781736,192.9240,0.020,0.010368,NaN
2,AAPL,2023-12-27,192.9040,0.100,0.051866,192.2450,-0.559,-0.289932,193.2530,0.449,0.232879,190.8470,-1.957,-1.015020,NaN
3,AAPL,2023-12-26,192.8040,-0.549,-0.283937,193.3630,0.010,0.005172,193.6430,0.290,0.149985,192.5840,-0.769,-0.397718,NaN
4,AAPL,2023-12-22,193.3530,-1.079,-0.554950,194.9310,0.499,0.256645,195.1610,0.729,0.374938,192.7240,-1.708,-0.878456,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80517,WMT,2013-01-08,54.1085,NaN,NaN,53.8640,NaN,NaN,54.3452,NaN,NaN,53.8087,NaN,NaN,NaN
80518,WMT,2013-01-07,53.9586,NaN,NaN,54.2978,NaN,NaN,54.3688,NaN,NaN,53.7417,NaN,NaN,NaN
80519,WMT,2013-01-04,54.4793,NaN,NaN,54.2190,NaN,NaN,54.5739,NaN,NaN,53.9113,NaN,NaN,NaN
80520,WMT,2013-01-03,54.2742,NaN,NaN,54.6292,NaN,NaN,54.7081,NaN,NaN,53.9428,NaN,NaN,NaN


In [3]:
# drop all change columns
combined_df_sorted = combined_df.drop(columns = ['Change', '% Change', 'Change.1', '% Change.1', 'Change.2', '% Change.2', 'Change.3', '% Change.3'])

In [4]:
# create a volatility column 
combined_df_sorted['Volatility'] = combined_df_sorted['PX_HIGH'] - combined_df_sorted['PX_LOW']

# create a change column based on the closing price of the day and the closing price of the date before
combined_df_sorted['Change'] = combined_df_sorted['PX_LAST'] - combined_df_sorted['PX_LAST'].shift(1)

# create a % change column based on the closing price
combined_df_sorted['% Change'] = (combined_df_sorted['PX_LAST'] - combined_df_sorted['PX_LAST'].shift(1)) / combined_df_sorted['PX_LAST'].shift(1) * 100

combined_df_sorted.head()

,stock,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,Volatility,Change,% Change
0,AAPL,2023-12-29,192.285,193.653,194.152,191.481,NaN,2.671,NaN,NaN
1,AAPL,2023-12-28,193.333,193.893,194.412,192.924,NaN,1.488,1.048,0.545024
2,AAPL,2023-12-27,192.904,192.245,193.253,190.847,NaN,2.406,-0.429,-0.221897
3,AAPL,2023-12-26,192.804,193.363,193.643,192.584,NaN,1.059,-0.100,-0.051839
4,AAPL,2023-12-22,193.353,194.931,195.161,192.724,NaN,2.437,0.549,0.284745
